In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt
import time

sys.path.insert(0, "/home/ylu/project/Benchmark_alignment_methods/scripts/methods")
from my_STAlign import stalign_align

%load_ext autoreload
%autoreload 2

In [3]:
method = "STAlign_whole"
data_folder = "/home/ylu/project/MOSTA/data/"
results_folder = f"./results/{method}/"
figures_folder = f"./results/figures/{method}"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [4]:
## load the data
from tqdm import tqdm
slices = []
for i in tqdm(range(1, 14)):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f"E16.5_E2S{i}.MOSTA.h5ad")))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [01:13<00:00,  5.66s/it]


In [5]:
spatial_key = "spatial"
key_added = "aligned_spatial"

In [ ]:
## Perform the STAlign alignment
import torch

# for i in tqdm(range(len(slices)-1)):
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    if i >0:
        previous_results = np.load(os.path.join(results_folder, f"slice_{i-1}_{i}.npy"), allow_pickle=True).item()
        slice2_spatial = previous_results['slice2_spatial']
        slice1.obsm[spatial_key] = slice2_spatial.copy()
    time_start = time.time()

    align_slices = stalign_align(
        models = [slice1, slice2],
        spatial_key = spatial_key,
        key_added = key_added,
        device = "cuda:0",
        scale=20,
        zero_mean=True,
    )
    
    time_end = time.time()
    alignment_results = {'slice1_spatial':align_slices[0].obsm[key_added], 'slice2_spatial': align_slices[1].obsm[key_added], 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}.npy"), alignment_results, allow_pickle=True)

  0%|                                                                                                                                                                                                                                 | 0/12 [00:00<?, ?it/s]/home/ylu/project/STalign/STalign/STalign.py:1044: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  L = torch.tensor(L,device=device,dtype=dtype,requires_grad=True)
/home/ylu/project/STalign/STalign/STalign.py:1045: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  T = torch.tensor(T,device=device,dtype=dtype,requires_grad=True)
/home/ylu/anaconda3/envs/STAlign/lib/python3.9/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in a